<a href="https://colab.research.google.com/github/soporteunah/bookllmraggemini/blob/main/llm_rag_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U llama-index llama-index-llms-google-genai llama-index-embeddings-google-genai

In [2]:
import os
import nest_asyncio
from google.colab import userdata
from llama_index.core import Settings

# --- Importaciones NUEVAS (Google GenAI) ---
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding

nest_asyncio.apply()

# Obtener API Key
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except:
    # Si no usas los secretos de Colab, pega tu clave aquí abajo:
    GOOGLE_API_KEY = "TU_CLAVE_AQUI"

# --- Configuración del Modelo (LLM) ---
# Usamos GoogleGenAI en lugar de la clase Gemini antigua.
# IMPORTANTE: Cambié "2.5" a "1.5-pro" porque el 2.5 no existe públicamente aún.
Settings.llm = GoogleGenAI(
    model_name="models/gemini-2.5-pro",
    api_key=GOOGLE_API_KEY,
    temperature=0.1
)

# --- Configuración de Embeddings ---
# Usamos GoogleGenAIEmbedding
Settings.embed_model = GoogleGenAIEmbedding(
    model_name="models/gemini-embedding-001",
    api_key=GOOGLE_API_KEY
)

print("¡Configuración de Google GenAI completada con éxito!")

¡Configuración de Google GenAI completada con éxito!


In [3]:
# 1. Descargar el archivo PDF
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

# 2. Cargar el documento
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

# 3. Dividir el texto en "nodos" (chunks de 1024 caracteres)
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

print(f"Documento cargado. Se han creado {len(nodes)} nodos.")

--2026-02-16 20:16:12--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 34.57.44.88
Connecting to openreview.net (openreview.net)|34.57.44.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: ‘metagpt.pdf’

metagpt.pdf         100%[===================>]  16.13M  38.0MB/s    in 0.4s    

2026-02-16 20:16:13 (38.0 MB/s) - ‘metagpt.pdf’ saved [16911937/16911937]

Documento cargado. Se han creado 34 nodos.


In [4]:
from llama_index.core import SummaryIndex, VectorStoreIndex

# Importante: Al cambiar el modelo de embeddings, el índice se regenera.
print("Creando índices con el nuevo modelo...")

# Crear índice de resumen
summary_index = SummaryIndex(nodes)

# Crear índice vectorial (Ahora sí debería funcionar)
vector_index = VectorStoreIndex(nodes)

print("¡ÉXITO! Índices creados correctamente.")

Creando índices con el nuevo modelo...
¡ÉXITO! Índices creados correctamente.


In [5]:
from llama_index.core.tools import QueryEngineTool

# 1. Motor de Resumen
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True
)

# 2. Motor Vectorial
vector_query_engine = vector_index.as_query_engine()

# 3. Empaquetar como Herramientas (Tools) con descripciones
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description="Useful for summarization questions related to MetaGPT"
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context from the MetaGPT paper."
)

print("Herramientas configuradas.")

Herramientas configuradas.


In [6]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

# Crear el Router
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True # Esto nos mostrará en pantalla qué decisión toma el modelo
)

print("Router Query Engine listo para usar.")

Router Query Engine listo para usar.


In [7]:
response = query_engine.query("What is the summary of the document?")
print("\n--- Respuesta ---")
print(str(response))

Selecting query engine 0: The question asks for a summary of the document, and choice 1 explicitly states it is useful for summarization questions..

--- Respuesta ---
MetaGPT is a meta-programming framework that uses Standardized Operating Procedures (SOPs) to enhance problem-solving in multi-agent systems using Large Language Models (LLMs). It assigns specialized roles to agents and uses an assembly line approach to break down complex tasks. The framework incorporates efficient human workflows into LLM-based multi-agent collaborations, allowing agents to verify intermediate results and reduce errors. MetaGPT achieves state-of-the-art performance on software engineering benchmarks by generating coherent solutions. It offers role definition, message sharing, and a novel executive feedback mechanism for debugging and code execution. The framework aims to introduce human practice into multi-agent frameworks and regulate LLM-based multi-agent systems.



In [8]:
response = query_engine.query("How do agents share information with other agents?")
print("\n--- Respuesta ---")
print(str(response))

Selecting query engine 1: The question 'How do agents share information with other agents?' requires retrieving specific details about the MetaGPT framework's implementation. Choice (2) is designed for retrieving specific context from the MetaGPT paper, making it the more relevant option..

--- Respuesta ---
Agents share information by publishing structured messages to a shared message pool where other agents can access them. Agents can also subscribe to specific information based on their role profiles, allowing them to receive only task-related information.



In [9]:
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings

# 1. Definir funciones normales de Python
def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int:
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)

# 2. Convertirlas en "Herramientas" para la IA
add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

# 3. Probar: Le pedimos a Gemini que use la función "mystery"
# Usamos predict_and_call, que hace que el LLM decida qué función ejecutar y con qué números.
response = Settings.llm.predict_and_call(
    [add_tool, mystery_tool],
    "Tell me the output of the mystery function on 2 and 9",
    verbose=True
)

print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"y": 9, "x": 2}
=== Function Output ===
121
121


In [10]:
from typing import List
from llama_index.core.vector_stores import FilterCondition, MetadataFilters

# 1. Definir la función de búsqueda con filtros
def vector_query(query: str, page_numbers: List[str]) -> str:
    """
    Perform a vector search over an index, filtering by specific page numbers.

    Args:
        query (str): The string query to be embedded.
        page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to search all pages.
    """

    # Crear los filtros de metadatos basados en los números de página
    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]

    # Configurar el motor de consulta con esos filtros
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )

    # Ejecutar la consulta y devolver la respuesta
    response = query_engine.query(query)
    return str(response)

# 2. Convertir la función en una Herramienta
vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

print("Herramienta de búsqueda por página creada.")

Herramienta de búsqueda por página creada.


In [11]:
# Gemini debería llamar a vector_query con page_numbers=['2']
response = Settings.llm.predict_and_call(
    [vector_query_tool],
    "What are the high-level results of MetaGPT as described on page 2?",
    verbose=True
)

print("\n--- Respuesta ---")
print(str(response))

# Verificar que realmente usó la página 2 revisando los metadatos (si la respuesta trajo nodos)
# Nota: predict_and_call a veces devuelve solo texto, depende de la versión.

=== Calling Function ===
Calling function: vector_tool with args: {"query": "high-level results of MetaGPT", "page_numbers": ["2"]}
=== Function Output ===
MetaGPT achieves a new state-of-the-art with 85.9% and 87.7% in Pass@1 in code generation benchmarks. It also achieves a 100% task completion rate.


--- Respuesta ---
MetaGPT achieves a new state-of-the-art with 85.9% and 87.7% in Pass@1 in code generation benchmarks. It also achieves a 100% task completion rate.



In [12]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

# 1. Asegurarnos de tener el summary_tool listo (como en la lección 1)
summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True
)

summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description="Useful if you want to get a summary of MetaGPT"
)

# 2. Prueba A: Pregunta Específica con página (Debería usar vector_tool)
print("--- PRUEBA 1: Pregunta específica (Página 8) ---")
response = Settings.llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "What are the MetaGPT comparisons with ChatDev described on page 8?",
    verbose=True
)
print(str(response))

# 3. Prueba B: Pregunta General (Debería usar summary_tool)
print("\n--- PRUEBA 2: Resumen general ---")
response = Settings.llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "What is a summary of the paper?",
    verbose=True
)
print(str(response))

--- PRUEBA 1: Pregunta específica (Página 8) ---
=== Calling Function ===
Calling function: vector_tool with args: {"page_numbers": ["8"], "query": "MetaGPT comparisons with ChatDev"}
=== Function Output ===
MetaGPT surpasses ChatDev on the SoftwareDev dataset across almost all metrics. It achieves an executability score of 3.75, which is close to flawless, and requires less time (503 seconds). MetaGPT also outperforms ChatDev in code statistics and human revision cost. While MetaGPT uses more tokens (24,613 or 31,255 versus 19,292), it needs fewer tokens per line of code (126.5/124.3 versus 248.9).

MetaGPT surpasses ChatDev on the SoftwareDev dataset across almost all metrics. It achieves an executability score of 3.75, which is close to flawless, and requires less time (503 seconds). MetaGPT also outperforms ChatDev in code statistics and human revision cost. While MetaGPT uses more tokens (24,613 or 31,255 versus 19,292), it needs fewer tokens per line of code (126.5/124.3 versus 2

ADAPTAR RAG CON GEMINI PRO


In [21]:
# 1. Importamos la clase NUEVA que aparece en tu captura (Línea 2)
from llama_index.core.agent import FunctionAgent
from llama_index.core.tools import QueryEngineTool
from llama_index.core import Settings

# 2. Aseguramos las herramientas (como siempre)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_index.as_query_engine(),
    description="Useful for retrieving specific context from the MetaGPT paper.",
    name="vector_tool"
)

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_index.as_query_engine(response_mode="tree_summarize"),
    description="Useful for summarization questions related to MetaGPT",
    name="summary_tool"
)

# 3. Creamos el Agente de la forma MODERNA
# Nota: Ya no se usa .from_tools(), se usa el constructor directo.
# Tampoco necesitamos 'Worker' ni 'Runner', el FunctionAgent ya lo hace todo.

agent = FunctionAgent(
    llm=Settings.llm,
    tools=[vector_tool, summary_tool],
    system_prompt="Eres un asistente útil capaz de usar herramientas para responder preguntas sobre documentos.",
    verbose=True
)

print("✅ ¡Agente Moderno (v0.13) creado con éxito!")

✅ ¡Agente Moderno (v0.13) creado con éxito!


In [23]:
import nest_asyncio
nest_asyncio.apply()

print("🤖 El Agente (Workflow) está pensando...")

# EN LA VERSIÓN NUEVA (v0.13+):
# 1. No existe .chat(), se usa .run()
# 2. Es asíncrono, por eso usamos 'await'
# 3. No devuelve un objeto 'Response' clásico, devuelve un evento de salida.

result = await agent.run(
    "Tell me about the agent roles in MetaGPT, and then how they communicate with each other."
)

print("\n--- Respuesta Final ---")
print(str(result))

🤖 El Agente (Workflow) está pensando...

--- Respuesta Final ---
MetaGPT employs five distinct roles within its software company structure: Product Manager, Architect, Project Manager, Engineer, and QA Engineer. These agents communicate by producing structured outputs such as documents and diagrams, utilizing a shared message pool for publishing and accessing information. Agents subscribe to pertinent information based on their roles, which helps them avoid information overload.



In [25]:
from llama_index.core.llms import ChatMessage, MessageRole

print("🧠 INICIO DE LA PRUEBA CON MEMORIA EXPLÍCITA\n")

# 1. Creamos el "Diario" (Historial vacío al principio)
chat_history = []

# --- TURNO 1 ---
pregunta1 = "What are the 5 agent roles defined in MetaGPT?"
print(f"👤 Usuario: {pregunta1}")

# AQUI ESTA EL TRUCO: Le pasamos el 'chat_history' al agente
response1 = await agent.run(pregunta1, chat_history=chat_history)
print(f"🤖 Agente: {response1}\n")

# ACTUALIZAMOS EL DIARIO MANUALMENTE
# Guardamos lo que dijo el usuario y lo que respondió el agente
chat_history.append(ChatMessage(role=MessageRole.USER, content=pregunta1))
chat_history.append(ChatMessage(role=MessageRole.ASSISTANT, content=str(response1)))

print("-" * 50 + "\n")

# --- TURNO 2 (La prueba de fuego) ---
pregunta2 = "Which of them is responsible for writing the code?"
print(f"👤 Usuario: {pregunta2}")

# Volvemos a llamar al agente, pero ahora 'chat_history' YA TIENE DATOS
response2 = await agent.run(pregunta2, chat_history=chat_history)
print(f"🤖 Agente: {response2}\n")

# (Opcional) Actualizamos el diario de nuevo por si queremos seguir hablando
chat_history.append(ChatMessage(role=MessageRole.USER, content=pregunta2))
chat_history.append(ChatMessage(role=MessageRole.ASSISTANT, content=str(response2)))

🧠 INICIO DE LA PRUEBA CON MEMORIA EXPLÍCITA

👤 Usuario: What are the 5 agent roles defined in MetaGPT?
🤖 Agente: MetaGPT defines five roles in its software company: Product Manager, Architect, Project Manager, Engineer, and QA Engineer.


--------------------------------------------------

👤 Usuario: Which of them is responsible for writing the code?
🤖 Agente: The Engineer role is responsible for writing the code in MetaGPT.




 (Agente sobre 11 documentos).

In [26]:
import nest_asyncio
nest_asyncio.apply()

# Lista de URLs y nombres de archivo
urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=LzPWWPAdY4",
    "https://openreview.net/pdf?id=VTF8yNQM66",
    "https://openreview.net/pdf?id=hSyW5go0v8",
    "https://openreview.net/pdf?id=9WD9KwssyT",
    "https://openreview.net/pdf?id=yv6fD7LYkF",
    "https://openreview.net/pdf?id=hnrB5YHoYu",
    "https://openreview.net/pdf?id=WbWtOYIzIK",
    "https://openreview.net/pdf?id=c5pwL0Soay",
    "https://openreview.net/pdf?id=TpD2aG1h0D"
]

papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "loftq.pdf",
    "swebench.pdf",
    "selfrag.pdf",
    "zipformer.pdf",
    "values.pdf",
    "finetune_fair_diffusion.pdf",
    "knowledge_card.pdf",
    "metra.pdf",
    "vr_mcl.pdf"
]

# Descargar los archivos
import requests
import os

print("⬇️ Descargando 11 papers científicos... (Esto puede tardar unos segundos)")
for url, paper in zip(urls, papers):
    if not os.path.exists(paper):
        !wget "{url}" -O "{paper}" -q
        print(f"✅ Descargado: {paper}")
    else:
        print(f"⚠️ Ya existe: {paper}")

⬇️ Descargando 11 papers científicos... (Esto puede tardar unos segundos)
⚠️ Ya existe: metagpt.pdf
✅ Descargado: longlora.pdf
✅ Descargado: loftq.pdf
✅ Descargado: swebench.pdf
✅ Descargado: selfrag.pdf
✅ Descargado: zipformer.pdf
✅ Descargado: values.pdf
✅ Descargado: finetune_fair_diffusion.pdf
✅ Descargado: knowledge_card.pdf
✅ Descargado: metra.pdf
✅ Descargado: vr_mcl.pdf


In [27]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex
from llama_index.core.tools import QueryEngineTool

def get_doc_tools(file_path: str, name: str):
    """Lee un PDF y devuelve una herramienta de vector y una de resumen."""

    # 1. Leer documento
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()

    # 2. Crear índices (Gemini Embeddings ya está configurado en Settings)
    vector_index = VectorStoreIndex.from_documents(documents)
    summary_index = SummaryIndex.from_documents(documents)

    # 3. Crear Motores
    vector_query_engine = vector_index.as_query_engine()
    summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize")

    # 4. Crear Herramientas con descripciones dinámicas
    vector_tool = QueryEngineTool.from_defaults(
        query_engine=vector_query_engine,
        description=f"Useful for retrieving specific context from the {name} paper.",
        name=f"vector_tool_{name}"
    )

    summary_tool = QueryEngineTool.from_defaults(
        query_engine=summary_query_engine,
        description=f"Useful for summarization questions related to {name}",
        name=f"summary_tool_{name}"
    )

    return vector_tool, summary_tool

In [30]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex, Document
from llama_index.core.tools import QueryEngineTool
from pathlib import Path
import os

# --- 1. Función de Limpieza (Igual que antes) ---
def clean_text(text: str) -> str:
    """Elimina caracteres extraños que rompen el encoder UTF-8"""
    if not text: return ""
    return text.encode('utf-8', 'ignore').decode('utf-8')

# --- 2. Función get_doc_tools CORREGIDA ---
def get_doc_tools(file_path: str, name: str):

    # Leer documento
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()

    cleaned_documents = []

    # CORRECCIÓN: En lugar de modificar doc.text, creamos un NUEVO Documento
    for doc in documents:
        safe_text = clean_text(doc.text)
        # Creamos una copia limpia manteniendo los metadatos
        new_doc = Document(text=safe_text, metadata=doc.metadata)
        cleaned_documents.append(new_doc)

    # Crear índices usando los documentos LIMPIOS
    vector_index = VectorStoreIndex.from_documents(cleaned_documents)
    summary_index = SummaryIndex.from_documents(cleaned_documents)

    # Crear Motores
    vector_query_engine = vector_index.as_query_engine()
    summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize")

    # Crear Herramientas
    vector_tool = QueryEngineTool.from_defaults(
        query_engine=vector_query_engine,
        description=f"Useful for retrieving specific context from the {name} paper.",
        name=f"vector_tool_{name}"
    )

    summary_tool = QueryEngineTool.from_defaults(
        query_engine=summary_query_engine,
        description=f"Useful for summarization questions related to {name}",
        name=f"summary_tool_{name}"
    )

    return vector_tool, summary_tool

# --- 3. Ejecutar el Bucle ---
paper_to_tools_dict = {}

print("⚙️ Procesando documentos (Intento final)...")

for paper in papers:
    # Verificación extra: Si el archivo está vacío (como values.pdf), lo saltamos
    if not os.path.exists(paper) or os.path.getsize(paper) == 0:
        print(f"⚠️ Saltando {paper} (Archivo vacío o no existe)")
        continue

    print(f"Processing: {paper}")
    try:
        stem_name = Path(paper).stem
        # Limpiamos el nombre para que sea válido como nombre de herramienta (sin puntos ni espacios)
        clean_stem_name = stem_name.replace(" ", "_").replace("-", "_")

        vector_tool, summary_tool = get_doc_tools(paper, clean_stem_name)
        paper_to_tools_dict[paper] = [vector_tool, summary_tool]
    except Exception as e:
        print(f"❌ Error crítico en {paper}: {e}")

# Aplanar la lista
all_tools = [t for paper in paper_to_tools_dict for t in paper_to_tools_dict[paper]]

print(f"\n🎉 ¡Éxito Total! Se han creado {len(all_tools)} herramientas.")

⚙️ Procesando documentos (Intento final)...
Processing: metagpt.pdf
Processing: longlora.pdf
Processing: loftq.pdf
Processing: swebench.pdf
Processing: selfrag.pdf
Processing: zipformer.pdf
⚠️ Saltando values.pdf (Archivo vacío o no existe)
Processing: finetune_fair_diffusion.pdf
Processing: knowledge_card.pdf
Processing: metra.pdf
Processing: vr_mcl.pdf

🎉 ¡Éxito Total! Se han creado 20 herramientas.


In [31]:
from llama_index.core.agent import FunctionAgent
from llama_index.core import Settings
import nest_asyncio

nest_asyncio.apply()

print("🤖 Creando el Agente Investigador con acceso a 10 Papers (20 Herramientas)...")

# Creamos el agente moderno (v0.13)
# Le damos la lista completa 'all_tools'
agent_super = FunctionAgent(
    llm=Settings.llm,
    tools=all_tools,
    system_prompt="""
    Eres un investigador de IA experto. Tienes acceso a una biblioteca de papers científicos.

    Tus instrucciones son:
    1. Para buscar detalles específicos (números, resultados), usa las herramientas 'vector_tool'.
    2. Para entender de qué trata un paper, usa las herramientas 'summary_tool'.
    3. Si te piden comparar, busca la información en los papers relevantes y contrasta los resultados.
    """,
    verbose=True
)

print("✅ ¡Agente listo y armado con conocimiento!")

🤖 Creando el Agente Investigador con acceso a 10 Papers (20 Herramientas)...
✅ ¡Agente listo y armado con conocimiento!


In [32]:
print("🧠 Ejecutando consulta compleja entre múltiples documentos...")

pregunta_final = """
Tell me about the evaluation dataset used in MetaGPT
and compare it against the one used in SWE-Bench.
Which one is more focused on software engineering tasks?
"""

# Usamos .run() con await
resultado = await agent_super.run(pregunta_final)

print("\n--- 🏆 RESPUESTA FINAL ---")
print(str(resultado))

🧠 Ejecutando consulta compleja entre múltiples documentos...

--- 🏆 RESPUESTA FINAL ---
MetaGPT uses HumanEval, MBPP, and SoftwareDev for evaluation. SWE-bench uses a dataset of software engineering tasks where the evaluation requires at least one test to change from fail to pass.

SWE-Bench is more focused on software engineering tasks.



## ADPTANDO A GRADIO PARA QUE SEA MAS AMIGABLE

In [33]:
!pip install gradio

In [1]:
!pip install llama-index-embeddings-huggingface

In [ ]:
import gradio as gr
import nest_asyncio
from pathlib import Path
import os
from google.colab import userdata

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex, Document, Settings
from llama_index.core.tools import QueryEngineTool
from llama_index.core.agent import FunctionAgent
from llama_index.llms.google_genai import GoogleGenAI
# CAMBIO: Usaremos HuggingFace para los embeddings (más estable)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.llms import ChatMessage, MessageRole

# 1. PARCHE ASYNCIO
nest_asyncio.apply()

# 2. API KEY (Solo para Gemini Chat, ya no para embeddings)
try:
    MY_API_KEY = userdata.get('GOOGLE_API_KEY')
except:
    MY_API_KEY = "TU_CLAVE_AQUI"

os.environ["GOOGLE_API_KEY"] = MY_API_KEY

# 3. CONFIGURACIÓN DE MODELOS (LA SOLUCIÓN)
try:
    # A) LLM: Usamos Gemini 1.5 Pro (Este sí funciona)
    Settings.llm = GoogleGenAI(
        model_name="models/gemini-1.5-pro",
        temperature=0,
        api_key=MY_API_KEY
    )

    # B) EMBEDDINGS: Usamos un modelo LOCAL multilingüe (Español/Inglés)
    # Esto elimina el error 404 para siempre.
    print("⬇️ Descargando modelo de embeddings local (esto toma unos segundos una sola vez)...")
    Settings.embed_model = HuggingFaceEmbedding(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    )
    print("✅ Modelos configurados: Gemini 1.5 Pro + HuggingFace Embeddings.")

except Exception as e:
    print(f"❌ Error fatal configurando modelos: {e}")

global_agent = None

def clean_text(text: str) -> str:
    if not text: return ""
    return text.encode('utf-8', 'ignore').decode('utf-8')

# --- 4. PROCESAMIENTO ---
def process_and_create_agent(files):
    global global_agent

    if not files:
        return "⚠️ Por favor, sube archivos primero."

    all_tools = []
    processed_names = []
    errors = []

    print(f"📂 Procesando {len(files)} archivos de Gobierno...")

    for file_path in files:
        try:
            filename = Path(file_path).name
            # Limpieza de nombre
            stem_name = Path(file_path).stem.replace(" ", "_").replace("-", "_")

            # 1. Cargar
            docs = SimpleDirectoryReader(input_files=[file_path]).load_data()

            # 2. Limpiar Texto
            cleaned_docs = [Document(text=clean_text(d.text), metadata=d.metadata) for d in docs]

            # 3. Indexar (Ahora usa HuggingFace, no fallará)
            vector_index = VectorStoreIndex.from_documents(cleaned_docs)
            summary_index = SummaryIndex.from_documents(cleaned_docs)

            # 4. Crear Tools
            vector_tool = QueryEngineTool.from_defaults(
                query_engine=vector_index.as_query_engine(),
                description=f"Useful for retrieving specific context from document: {filename}",
                name=f"vector_tool_{stem_name[:50]}" # Recortamos nombres muy largos
            )
            summary_tool = QueryEngineTool.from_defaults(
                query_engine=summary_index.as_query_engine(response_mode="tree_summarize"),
                description=f"Useful for summarization of document: {filename}",
                name=f"summary_tool_{stem_name[:50]}"
            )

            all_tools.extend([vector_tool, summary_tool])
            processed_names.append(filename)
            print(f"✅ {filename} OK")

        except Exception as e:
            print(f"❌ Error en {filename}: {e}")
            errors.append(f"{filename}: {str(e)}")
            continue

    if not all_tools:
        return f"❌ Error total. Detalles: {errors}"

    # Crear el Agente
    try:
        global_agent = FunctionAgent(
            llm=Settings.llm,
            tools=all_tools,
            system_prompt="Eres un experto en gestión pública y análisis de documentos. Responde siempre en español.",
            verbose=True
        )
    except Exception as e:
        return f"Error creando agente: {e}"

    return f"✅ ¡Listo! {len(processed_names)} documentos activos."

# --- 5. CHAT ---
async def chat_logic(message, history):
    global global_agent
    if global_agent is None:
        return "⚠️ Primero procesa los archivos."

    chat_history_objs = []
    for human_msg, ai_msg in history:
        chat_history_objs.append(ChatMessage(role=MessageRole.USER, content=human_msg))
        if ai_msg:
            chat_history_objs.append(ChatMessage(role=MessageRole.ASSISTANT, content=str(ai_msg)))

    try:
        response = await global_agent.run(message, chat_history=chat_history_objs)
        return str(response)
    except Exception as e:
        return f"Error en el chat: {str(e)}"

# --- INTERFAZ ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🇵🇪 Analista de Planes de Gobierno (Gemini + Local Embeddings)")

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(label="📂 Sube Planes de Gobierno (PDF)", file_count="multiple", file_types=[".pdf"])
            process_btn = gr.Button("⚙️ Procesar Documentos", variant="primary")
            status_box = gr.Textbox(label="Estado", interactive=False)

        with gr.Column(scale=2):
            gr.ChatInterface(
                fn=chat_logic,
                chatbot=gr.Chatbot(height=600),
                textbox=gr.Textbox(placeholder="Ej: Compara el plan de Electroperu con el del Ministerio de Defensa...", container=False, scale=7),
                title="💬 Chat"
            )

    process_btn.click(process_and_create_agent, inputs=[file_input], outputs=[status_box])

demo.launch(debug=True)